# RESUMO

## Sobre a pesquisa Pense 2015

A base de dados estudada nesse módulo é do IBGE, chamada Pense (Pesquisa Nacional da Saúde do Escolar). Podemos entender, através dela, a relação entre saúde e educação e como o ambiente de vivência do estudante pode influenciar na qualidade de seu desenvolvimento escolar e vice-versa.

Essa pesquisa foi feita em 2009 e 2012 para alunos do 9° ano. Na edição de 2015, importantes inovações foram introduzidas na pesquisa, dentre as quais se destaca a disponibilização de informações oriundas de dois planos amostrais distintos: escolares frequentando o 9o ano do ensino fundamental e escolares de 13 a 17 anos de idade frequentando as etapas do 6o ao 9o ano do ensino fundamental (antigas 5a a 8a séries) e da 1a a 3a série do ensino médio.

## Sobre a análise

Será feita uma EDA (Exploratory Data Analysis) sobre a pesquisa Pense 2015.

Inicialmente, essa análise será feita sobre algumas variáveis específicas, para conhecermos seus dados e suas distribuições, como peso, altura, IMC, sexo, raça e outras. 

A análise principal será voltada para o entendimento da relação entre idade, sexo e raça com o o IMC e a satisfação dos alunos com o corpo, verificando quais variáveis podem influenciar nesse item e como o próprio IMC pode influenciar nessa satisfação corporal.

## Links e fontes

* Link da pesquisa: https://www.ibge.gov.br/estatisticas/sociais/educacao/9134-pesquisa-nacional-de-saude-do-escolar.html?=&t=o-que-e
* Link do dataset: https://www.ibge.gov.br/estatisticas/sociais/educacao/9134-pesquisa-nacional-de-saude-do-escolar.html?=&t=microdados
* Especificações de tabelas e indicadores Amostra2 : https://ftp.ibge.gov.br/pense/2015/microdados/Notas_Metodologicas/Nota_metodologica_03_especificacao_tabelas_Amostra_2_20180514.pdf

# IMPORTS, DADOS E FUNÇÕES AUXILIARES

## Imports de bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot
from scipy.stats import norm
import datetime
from datetime import date
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Image, HTML

import plotly.io as pio
pio.renderers

pio.renderers.default = "svg"

## Carregamento dos dados

In [2]:
dados = pd.read_csv('dados/PENSE_AMOSTRA2_ALUNO.CSV', sep = ';', thousands = '.', decimal = ',')

In [3]:
dados.sample(5)

,ANOPESQ,PAIS,REGEOGR,VB00004,VB01001,VB01002,VB01003,VB01004,VB01005,VB01006,...,ESTRATO_EXP,ESTRATOGEOREG,PESO,V0006,V0007,V0008,V0041,aluno,escola,turma
15276,2015,76,5,1,2,4,11,8,9,1,...,5121,5,169.245582,1,1,2,-1,15277,347,39
110,2015,76,1,1,2,4,13,10,7,1,...,1221,1,86.718961,1,2,4,1,111,2,278
4341,2015,76,2,1,2,1,11,2,10,2,...,2213,2,585.887643,1,2,4,1,4342,102,108
6037,2015,76,2,1,1,5,12,4,9,1,...,2123,2,1085.028190,1,1,2,-1,6038,137,54
4430,2015,76,2,1,1,4,14,10,6,1,...,2122,2,1447.532105,1,1,1,-1,4431,104,60


## Funções auxiliares

### Formatação de floats

In [4]:
def formata_float_df(df):
    for i in range(0, len(df.values)):
        df.values[i] = "%.2f" % df.values[i]
        df.values[i] = float(df.values[i])

# DESCRIÇÃO DOS DADOS

### Tamanho do dataset

In [5]:
print(f'Número de linhas do dataset: {dados.shape[0]}')
print(f'Número de colunas do dataset: {dados.shape[1]}')

Número de linhas do dataset: 16556
Número de colunas do dataset: 181


### Tipos de dados

In [6]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16556 entries, 0 to 16555
Columns: 181 entries, ANOPESQ to turma
dtypes: float64(10), int64(171)
memory usage: 22.9 MB


# ESTATÍSTICA DESCRITIVA

### Raça

#### Número de respondentes de acordo com a raça

In [7]:
# lista com as labels
raca = ['Branca', 'Preta', 'Amarela', 'Parda', 'Indígena', 'Não informado']

# criando o dataframe que contem o número de respondentes sobre a raça
raca_abs = pd.DataFrame(dados['VB01002'].value_counts().sort_index())
raca_abs.index = raca
raca_abs.index.name = 'Raça'
raca_abs.columns = ['Nº de respondentes']

raca_abs

,Nº de respondentes
Raça,
Branca,6575
Preta,1939
Amarela,712
Parda,6726
Indígena,581
Não informado,23


In [ ]:
fig = px.bar(data_frame = raca_abs, x = raca_abs.index, y = 'Nº de respondentes',
               color = raca_abs.index, color_discrete_sequence=px.colors.qualitative.Dark2,
               title = 'Número de respondentes, de acordo com a raça')
fig.update_layout(font_family="Rockwell", legend=dict(
                  title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"))
fig.show()


#### Porcentagens de respondentes de acordo com a raça

In [ ]:
fig = px.pie(raca_abs,
             values='Nº de respondentes',
             names= raca_abs.index,
             labels = raca_abs.index,
             title='Números e porcentagens de respondentes de acordo com a raça')
fig.show()

### Sexo

#### Número de respondentes de acordo com o sexo

In [ ]:
# criação do dataframe
resp_por_sexo = pd.DataFrame(dados['VB01001'].value_counts())
resp_por_sexo.index = ['Masculino', 'Feminino']
resp_por_sexo.index.name = 'Sexo'
resp_por_sexo.columns = ['Número de respondentes']
resp_por_sexo

In [ ]:
fig = px.bar(data_frame = resp_por_sexo, x = resp_por_sexo.index, y = 'Número de respondentes',
               color = resp_por_sexo.index, color_discrete_sequence=px.colors.qualitative.Dark2,
               title = 'Número de respondentes, de acordo com o sexo')
fig.update_layout(font_family="Rockwell", legend=dict(
                  title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"))
fig.show()


A quantidade de alunos, respondentes da pesquisa, de cada sexo foi extremamente equilibrada.

#### Porcentagens de respondentes de acordo com o sexo

In [ ]:
fig = px.pie(resp_por_sexo, values = 'Número de respondentes', names = resp_por_sexo.index,
             labels = resp_por_sexo.index, title = 'Porcentagem de respondentes por sexo')
fig.show()

### Peso

#### Estatísticas dos <ins>pesos</ins>

Média

In [ ]:
media_pesos = dados['VB17003'].mean()
print(f'Média dos pesos dos alunos: {media_pesos}')

Mediana

In [ ]:
mediana_pesos = dados['VB17003'].median()
print(f'Média dos pesos dos alunos: {mediana_pesos}')

Moda

In [ ]:
moda_pesos = dados['VB17003'].mode()[0]
print(f'Moda dos pesos dos alunos: {moda_pesos}') 

Desvio Padrão

In [ ]:
desvio_padrao_pesos = dados['VB17003'].std()
print(f'Média dos pesos dos alunos: {desvio_padrao_pesos}')

#### Estatísticas dos <ins>pesos</ins> X <ins>classe de idade</ins>

Criando a coluna CLASSE_IDADE 

In [ ]:
dados['CLASSE_IDADE'] = pd.cut(x = dados['VB01003'],
                               bins = np.histogram_bin_edges(dados['VB01003'], bins = 4),
                               include_lowest=True)
dados['CLASSE_IDADE'] = dados['CLASSE_IDADE'].astype(str)

In [ ]:
dados['CLASSE_IDADE'] = dados['CLASSE_IDADE'].astype(str)

Média dos pesos por classe de idade

In [ ]:
media_peso_classes_idades = pd.DataFrame(dados.groupby('CLASSE_IDADE')['VB17003'].mean())
media_peso_classes_idades.columns = ['Média dos pesos']
media_peso_classes_idades.index.name = 'Classes das Idades'

formata_float_df(media_peso_classes_idades)
media_peso_classes_idades

Mediana dos pesos por classe de idade

In [ ]:
mediana_pesos_por_classe_idade = pd.DataFrame(dados.groupby('CLASSE_IDADE')['VB17003'].median())
mediana_pesos_por_classe_idade.columns = ['Mediana dos pesos']
mediana_pesos_por_classe_idade.index.name = 'Classes das Idades'

mediana_pesos_por_classe_idade

Moda dos pesos por classe de idade

In [ ]:
lista_valores_classe_idade = dados['CLASSE_IDADE'].unique()
len_valores_classe_idade = len(lista_valores_classe_idade)
lista = []
for i in range(0, len_valores_classe_idade):
    lista.append(dados['VB17003'][dados['CLASSE_IDADE'] == lista_valores_classe_idade[i]].mode()[0])
modas_classe_idade = pd.DataFrame({'Moda': lista}, index = lista_valores_classe_idade)
modas_classe_idade.index.name = 'Classes das idades'
modas_classe_idade

Desvio padrão dos pesos por classe de idade

In [ ]:
desvios_padrao_pesos_por_classe_idade = pd.DataFrame(dados.groupby('CLASSE_IDADE')['VB17003'].std())
desvios_padrao_pesos_por_classe_idade.columns = ['Desvio padrão dos pesos']
desvios_padrao_pesos_por_classe_idade.index.name = 'Classes das idades'
desvios_padrao_pesos_por_classe_idade

#### Estatísticas dos <ins>pesos</ins> X <ins>sexo e classe de idade</ins>

Média dos pesos de acordo com o sexo e classe de idade

In [ ]:
crosstab_media_peso_sexo_idade = pd.crosstab(index = dados['CLASSE_IDADE'],
                                       columns=dados['VB01001'],
                                       values = dados['VB17003'],
                                       aggfunc='mean')

crosstab_media_peso_sexo_idade.index.name = 'Classes das idades'
crosstab_media_peso_sexo_idade.columns.name = ''
crosstab_media_peso_sexo_idade.columns = ['Masculino', 'Feminino']
print('Média dos pesos por classe de idade e sexo: ')
crosstab_media_peso_sexo_idade

Mediana dos pesos de acordo com o sexo e classe de idade

In [ ]:
crosstab_mediana_peso_sexo_idade = pd.crosstab(index = dados['CLASSE_IDADE'],
                                       columns=dados['VB01001'],
                                       values = dados['VB17003'],
                                       aggfunc='median')

crosstab_mediana_peso_sexo_idade.index.name = 'Classes das idades'
crosstab_mediana_peso_sexo_idade.columns.name = ''
crosstab_mediana_peso_sexo_idade.columns = ['Masculino', 'Feminino']
print('Mediana dos pesos por classe de idade e sexo: ')
crosstab_mediana_peso_sexo_idade

Moda dos pesos de acordo com o sexo e classe de idade

In [ ]:
valores_classes_idade = dados['CLASSE_IDADE'].unique()
valores_sexo = dados['VB01001'].sort_values().unique()
lista_masculino = []
lista_feminino = []

for classe in valores_classes_idade:
    for sexo in valores_sexo:
        if sexo == 1:
            lista_masculino.append(dados['VB17003'][(dados['VB01001'] == sexo) & (dados['CLASSE_IDADE'] == classe)].mode()[0])
        elif sexo == 2:
            lista_feminino.append(dados['VB17003'][(dados['VB01001'] == sexo) & (dados['CLASSE_IDADE'] == classe)].mode()[0])

In [ ]:
df_moda_peso_sexo_idade = pd.DataFrame([lista_masculino, lista_feminino]).T
df_moda_peso_sexo_idade.columns = ['Masculino', 'Feminino']
df_moda_peso_sexo_idade.index = valores_classes_idade
df_moda_peso_sexo_idade

##### Desvios padrão dos pesos de acordo com o sexo e com a classe de idade

In [ ]:
desvio_padrao_pesos_sexo_idade = pd.crosstab(index = dados['CLASSE_IDADE'],
            columns = dados['VB01001'],
            values = dados['VB17003'],
            aggfunc='std')
desvio_padrao_pesos_sexo_idade.columns = ['Masculino', 'Feminino']
desvio_padrao_pesos_sexo_idade.index.name = 'Classes das idades'
print('Desvios padrão dos pesos de acordo com o sexo e com a classe de idade:')
desvio_padrao_pesos_sexo_idade

#### Alguns gráficos

##### Histograma - Distribuição geral dos pesos

In [ ]:
ax = sns.histplot(data = dados, x = 'VB17003')
ax.figure.set_size_inches((13, 6))
ax.set_title('Distribuição dos pesos', fontsize = 22)
ax.set_xlabel('Pesos', fontsize = 12)
ax.set_ylabel('Quantidade de ocorrências', fontsize = 12)

# linha com a média dos pesos
ax.axvline(x = dados['VB17003'].mean(),
           label = 'Média',
           linewidth = 2,
           color = '#000000')
# linha com a mediana dos pesos
ax.axvline(x = dados['VB17003'].median(),
           label = 'Mediana',
           linewidth = 2,
           linestyle = '--',
           color = '#7FFF00')
#linha com a moda dos pesos
ax.axvline(x = dados['VB17003'].mode().values,
           label = 'Moda',
           linewidth = 2,
           linestyle = '-.',
           color = '#FF1493')

# intervalo de confiança da distribuição dos pesos
xmin = media_pesos - desvio_padrao_pesos
xmax = media_pesos + desvio_padrao_pesos
# área do desvio padrão
ax.axvspan(xmin, xmax, alpha=0.3, color='gray', label='Desvio Padrão')

ax.legend(title = 'Medidas')
ax;

##### Boxplot - distribuição dos pesos de acordo com a classe de idade

In [ ]:
ax = sns.boxplot(x = dados['CLASSE_IDADE'], y = dados['VB17003'], palette = 'winter_r')
ax.figure.set_size_inches((13,6))
ax.set_xlabel('Classe de idade', fontsize = 14)
ax.set_ylabel('Peso', fontsize = 14)
ax;

##### Boxplot - Distribuição dos pesos de acordo com o sexo e classe de idade

In [ ]:
ax = sns.boxplot(data = dados,
                 x = 'CLASSE_IDADE',
                 y = 'VB17003',
                 hue = 'VB01001', palette = 'Accent_r')
ax.figure.set_size_inches((13,6))
ax.set_xlabel('Classe de idade', fontsize = 14)
ax.set_ylabel('Peso', fontsize = 14)
handles, _ = ax.get_legend_handles_labels()          
ax.legend(handles, ["Masculino", "Feminino"], loc="best")
ax;

### Altura

#### Estatísticas da <ins>altura</ins>

Média

In [ ]:
media_altura = dados['VB17004'].mean()
print(f'Média da altura: {media_altura}')

Mediana

In [ ]:
mediana_altura = dados['VB17004'].median()
print(f'Média da altura: {mediana_altura}')

Moda

In [ ]:
moda_altura = dados['VB17004'].mode()[0]
print(f'Média da altura: {moda_altura}')

Desvio Padrão

In [ ]:
desvio_padrao_altura = dados['VB17004'].std()
print(f'Média da altura: {desvio_padrao_altura}')

##### Unificação das estatísticas (dataframe)

In [ ]:
df_estatisticas_altura = pd.DataFrame([media_altura, mediana_altura, moda_altura,
                                       desvio_padrao_altura],
             index = ['Media', 'Mediana', 'Moda', 'Desvio Padrão'])
formata_float_df(df_estatisticas_altura)
df_estatisticas_altura.columns = ['']
df_estatisticas_altura

##### Histograma - Distribuição da altura 

In [ ]:
ax = sns.histplot(data = dados, x = 'VB17004')
ax.figure.set_size_inches((13, 6))
ax.set_title('Distribuição da altura', fontsize = 22)
ax.set_xlabel('Altura', fontsize = 12)
ax.set_ylabel('Quantidade de ocorrências', fontsize = 12)

# linha com a média para o histograma
ax.axvline(x = dados['VB17004'].mean(), linestyle = '-', linewidth = 2.5,
           label = 'Média',
           color = '#000000')

# linha com a mediana para o histograma
ax.axvline(x = dados['VB17004'].median(), linestyle = '--', linewidth = 2.5,
           label = 'Mediana',
           color = '#7FFF00')

# linha com a moda para o histograma
ax.axvline(x = dados['VB17004'].mode()[0], linestyle = '-.', linewidth = 2.5,
           label = 'Mediana',
           color = '#FF1493')

# intervalo de confiança da distribuição dos pesos
xmin = media_altura - desvio_padrao_altura
xmax = media_altura + desvio_padrao_altura
# área do desvio padrão
ax.axvspan(xmin, xmax, alpha=0.3, color='gray', label='Desvio Padrão')

ax.legend(title = 'Medidas')
ax;

#### Estatísticas da <ins>altura</ins>  X  <ins>classe de idade</ins>

Médias das alturas por classe de idade

In [ ]:
media_alturas_por_classe_idade = pd.DataFrame(dados.groupby('CLASSE_IDADE')['VB17004'].mean())
media_alturas_por_classe_idade.index.name = 'Classe da idade'
media_alturas_por_classe_idade.columns = ['Média da altura por classe de idade']
media_alturas_por_classe_idade

Medianas das alturas por classe de idade

In [ ]:
mediana_alturas_por_classe_idade = pd.DataFrame(dados.groupby('CLASSE_IDADE')['VB17004'].median())
media_alturas_por_classe_idade.index.name = 'Classe da idade'
media_alturas_por_classe_idade.columns = ['Mediana da altura por classe de idade']
mediana_alturas_por_classe_idade

Modas das alturas por classe de idade

In [ ]:
lista_valores_classe_idade = dados['CLASSE_IDADE'].unique()
len_valores_classe_idade = len(lista_valores_classe_idade)
lista = []
for i in range(0, len_valores_classe_idade):
    lista.append(dados['VB17004'][dados['CLASSE_IDADE'] == lista_valores_classe_idade[i]].mode()[0])
df_modas_classe_idade = pd.DataFrame({'Moda': lista}, index = lista_valores_classe_idade)
df_modas_classe_idade.index.name = 'Classe (idade)'
df_modas_classe_idade

Desvios padrão das alturas por classe de idade

In [ ]:
desvios_padrao_altura_por_classe_idade = pd.DataFrame(dados.groupby('CLASSE_IDADE')['VB17004'].std())
desvios_padrao_altura_por_classe_idade.index.name = 'Classe de idade'
desvios_padrao_altura_por_classe_idade.columns = ['Desvio padrão por classe de idade']
desvios_padrao_altura_por_classe_idade

#### Estatísticas das <ins>alturas</ins>  X  <ins>sexo e classe de idade</ins>

Valores mínimo e máximo das alturas X sexo e classe de idade

In [ ]:
minimo_altura_sexo_idade = pd.crosstab(index = dados['CLASSE_IDADE'],
            columns = dados['VB01001'],
            values = dados['VB17004'],
            aggfunc = 'min')
minimo_altura_sexo_idade.index.name = 'Classes de idade'
minimo_altura_sexo_idade.columns.name = ''
minimo_altura_sexo_idade.columns = ['Masculino', 'Feminino']
print('Valores mínimos das alturas por sexo e idade:')
minimo_altura_sexo_idade

In [ ]:
maximo_altura_sexo_idade = pd.crosstab(index = dados['CLASSE_IDADE'],
            columns = dados['VB01001'],
            values = dados['VB17004'],
            aggfunc = 'max')
maximo_altura_sexo_idade.index.name = 'Classes de idade'
maximo_altura_sexo_idade.columns.name = ''
maximo_altura_sexo_idade.columns = ['Masculino', 'Feminino']
print('Valores máximos das alturas por sexo e idade:')
maximo_altura_sexo_idade

Média das alturas de acordo com o sexo e a classe de idade

In [ ]:
media_alturas_sexo_idade = pd.crosstab(index = dados['CLASSE_IDADE'],
                                       columns = dados['VB01001'],
                                       values = dados['VB17004'],
                                       aggfunc = 'mean')
media_alturas_sexo_idade.columns = ['Masculino', 'Feminino']
media_alturas_sexo_idade.index.name = 'Classes das idades'
print('Médias das alturas por sexo e por idade')
media_alturas_sexo_idade

Mediana das alturas de acordo com o sexo e a classe de idade

In [ ]:
mediana_alturas_sexo_idade = pd.crosstab(index = dados['CLASSE_IDADE'],
                                       columns = dados['VB01001'],
                                       values = dados['VB17004'],
                                       aggfunc = 'median')
mediana_alturas_sexo_idade.columns = ['Masculino', 'Feminino']
mediana_alturas_sexo_idade.index.name = 'Classes das idades'
print('Medianas das alturas por sexo e por idade')
mediana_alturas_sexo_idade

Moda dos pesos de acordo com o sexo e classe de idade

In [ ]:
valores_classes_idade = dados['CLASSE_IDADE'].unique()
valores_sexo = dados['VB01001'].sort_values().unique()
lista_masculino = []
lista_feminino = []

for classe in valores_classes_idade:
    for sexo in valores_sexo:
        if sexo == 1:
            lista_masculino.append(dados['VB17004'][(dados['VB01001'] == sexo) & (dados['CLASSE_IDADE'] == classe)].mode()[0])
        elif sexo == 2:
            lista_feminino.append(dados['VB17004'][(dados['VB01001'] == sexo) & (dados['CLASSE_IDADE'] == classe)].mode()[0])

In [ ]:
df_moda_alturas_sexo_idade = pd.DataFrame([lista_masculino, lista_feminino]).T
df_moda_alturas_sexo_idade.index = [valores_classes_idade]
df_moda_alturas_sexo_idade.columns = ['Masculino', 'Feminino']
df_moda_alturas_sexo_idade

Desvios padrão das alturas de acordo com o sexo e a classe de idade

In [ ]:
desvio_padrao_alturas_sexo_idade = pd.crosstab(index = dados['CLASSE_IDADE'],
            columns = dados['VB01001'],
            values = dados['VB17004'],
            aggfunc='std')
desvio_padrao_alturas_sexo_idade.columns = ['Masculino', 'Feminino']
desvio_padrao_alturas_sexo_idade.index.name = 'Classes das idades'
desvio_padrao_alturas_sexo_idade

#### Alguns gráficos

##### Histograma - Distribuição geral das alturas

In [ ]:
ax = sns.histplot(data = dados, x = 'VB17004')
ax.figure.set_size_inches((13, 6))
ax.set_title('Distribuição das Altura', fontsize = 22)
ax.set_xlabel('Altura', fontsize = 12)
ax.set_ylabel('Quantidade de ocorrências', fontsize = 12)

# linha com a média dos pesos
ax.axvline(x = dados['VB17004'].mean(),
           label = 'Média',
           linewidth = 2,
           color = '#000000')
# linha com a mediana dos pesos
ax.axvline(x = dados['VB17004'].median(),
           label = 'Mediana',
           linewidth = 2,
           linestyle = '--',
           color = '#7FFF00')
#linha com a moda dos pesos
ax.axvline(x = dados['VB17004'].mode().values,
           label = 'Moda',
           linewidth = 2,
           linestyle = '-.',
           color = '#FF1493')

# intervalo de confiança da distribuição dos pesos
xmin = media_altura - desvio_padrao_altura
xmax = media_altura + desvio_padrao_altura
# área do desvio padrão
ax.axvspan(xmin, xmax, alpha=0.3, color='gray', label='Desvio Padrão')

ax.legend(title = 'Medidas')
ax;

##### Boxplot - distribuição das alturas de acordo com a classe de idade

In [ ]:
ax = sns.boxplot(x = dados['CLASSE_IDADE'], y = dados['VB17004'], palette = 'winter_r')
ax.figure.set_size_inches((13,6))
ax.set_xlabel('Classe de idade', fontsize = 14)
ax.set_ylabel('Altura', fontsize = 14)
ax;

##### Boxplot - Distribuição das alturas de acordo com o sexo e classe de idade

In [ ]:
ax = sns.boxplot(data = dados,
                 x = 'CLASSE_IDADE',
                 y = 'VB17004',
                 hue = 'VB01001', palette = 'Accent_r')
ax.figure.set_size_inches((13,6))
ax.set_xlabel('Classe de idade', fontsize = 14)
ax.set_ylabel('Altura', fontsize = 14)
handles, _ = ax.get_legend_handles_labels()          
ax.legend(handles, ["Masculino", "Feminino"], loc="best")
ax;

### IMC

#### Criação das colunas IMC

In [ ]:
dados['IMC'] = dados['VB17003'] / ((dados['VB17004']/100)**2)

#### Estatísticas sobre o <ins>IMC</ins>

Valores mínimo e máximo da coluna IMC

In [ ]:
min_max_imc = pd.DataFrame([dados['IMC'].min(), dados['IMC'].max()])
min_max_imc.index = ['Mínimo', 'Máximo']
min_max_imc.columns = ['Valores mínimo e máximo do IMC']
min_max_imc

Média dos IMCs 

In [ ]:
media_imc = dados['IMC'].mean()
print(f'Média dos IMCs: {media_imc}')

Mediana dos IMCs

In [ ]:
mediana_imc = dados['IMC'].median()
print(f'Mediana dos IMCs: {mediana_imc}')

Desvio Padrão dos IMCs

In [ ]:
desvio_padrao_imc = dados['IMC'].std()
print(f'Desvio Padrão dos IMCs: {desvio_padrao_imc}')

#### Estatísticas sobre o <ins>IMC</ins> X <ins>classe de idade</ins>

Valores mínimo e máximo dos IMCs x classe de idade

In [ ]:
minimo_imc_sexo_idade = dados.groupby('CLASSE_IDADE')['IMC'].min()
maximo_imc_sexo_idade = dados.groupby('CLASSE_IDADE')['IMC'].max()
min_max = pd.DataFrame([minimo_imc_sexo_idade, maximo_imc_sexo_idade]).T
min_max.index.name = 'Classes de idade'
min_max.columns = ['Mínimo', 'Máximo']
min_max

Podemos perceber no dataframe acima que o valor mínimo da classe de idade entre 10.999 e 13 anos é considerado um outlier. Há uma grande probabilidade de que o valor do peso ou da altura tenha sido preenchido de forma errada. Vamos conferir:

In [ ]:
df = dados[['IMC', 'VB17003', 'VB17004', 'VB01003', 'VB01001']][dados['IMC'] == 0.39786663908124625]
df.columns = ['IMC', 'Peso', 'Altura', 'Idade', 'Sexo']
df

Realmente: o valor do peso parece ter sido preenchido de maneira equivocada, pois o peso mínimo médio de um menino de 12 anos é de 30,48 kg, com altura mínima média de 138,1 cm.

Fonte: https://www.unimed.coop.br/web/vitoria/viver-bem/pais-e-filhos/estatura-por-idade

Média dos IMCs X classe de idade

In [ ]:
media_imc_classe_idade = pd.DataFrame(dados.groupby('CLASSE_IDADE')['IMC'].mean())
media_imc_classe_idade

Mediana dos IMCs X classe de idade

In [ ]:
mediana_imc_classe_idade = pd.DataFrame(dados.groupby('CLASSE_IDADE')['IMC'].median())
mediana_imc_classe_idade

Moda dos IMCs X classe de idade

In [ ]:
lista_valores_classe_idade = dados['CLASSE_IDADE'].unique()
len_valores_classe_idade = len(lista_valores_classe_idade)
lista = []
for i in range(0, len_valores_classe_idade):
    lista.append(dados['IMC'][dados['CLASSE_IDADE'] == lista_valores_classe_idade[i]].mode().values)
df_modas_classe_idade = pd.DataFrame({'Moda': lista}, index = lista_valores_classe_idade)
df_modas_classe_idade.index.name = 'Classe (idade)'
df_modas_classe_idade

Desvios padrão dos IMCs X classe de idade

In [ ]:
desvio_padrao_imc_idade = pd.DataFrame(dados.groupby('CLASSE_IDADE')['IMC'].std())
desvio_padrao_imc_idade

#### Estatísticas sobre o <ins>IMC</ins> X <ins>sexo e classe de idade</ins>

Valores mínimo e máximo dos IMCs x sexo e classe de idade

In [ ]:
minimo_imc_sexo_idade = pd.crosstab(index = dados['CLASSE_IDADE'],
            columns = dados['VB01001'],
            values = dados['IMC'],
            aggfunc = 'min')
minimo_imc_sexo_idade.index.name = 'Classes de idade'
minimo_imc_sexo_idade.columns.name = ''
minimo_imc_sexo_idade.columns = ['Masculino', 'Feminino']
print('Valores mínimos dos IMCs por sexo e idade:')
minimo_imc_sexo_idade

In [ ]:
maximo_imc_sexo_idade = pd.crosstab(index = dados['CLASSE_IDADE'],
            columns = dados['VB01001'],
            values = dados['IMC'],
            aggfunc = 'max')
maximo_imc_sexo_idade.index.name = 'Classes de idade'
maximo_imc_sexo_idade.columns.name = ''
maximo_imc_sexo_idade.columns = ['Masculino', 'Feminino']
print('Valores máximos dos IMCs por sexo e idade:')
maximo_imc_sexo_idade

Média dos IMCs X sexo e classe de idade

In [ ]:
media_imc_sexo_idade = pd.crosstab(index = dados['CLASSE_IDADE'],
            columns = dados['VB01001'],
            values = dados['IMC'],
            aggfunc = 'mean')
media_imc_sexo_idade.index.name = 'Classes de idade'
media_imc_sexo_idade.columns.name = ''
media_imc_sexo_idade.columns = ['Masculino', 'Feminino']
print('Médias dos IMCs por sexo e idade:')
media_imc_sexo_idade

Mediana dos IMCs X sexo e classe de idade

In [ ]:
media_imc_sexo_idade = pd.crosstab(index = dados['CLASSE_IDADE'],
            columns = dados['VB01001'],
            values = dados['IMC'],
            aggfunc = 'median')
media_imc_sexo_idade.index.name = 'Classes de idade'
media_imc_sexo_idade.columns.name = ''
media_imc_sexo_idade.columns = ['Masculino', 'Feminino']
print('Medianas dos IMCs por sexo e idade:')
media_imc_sexo_idade

Desvios padrão dos IMCs X sexo e classe de idade

In [ ]:
valores_classes_idade = dados['CLASSE_IDADE'].unique()
valores_sexo = dados['VB01001'].sort_values().unique()
lista_masculino = []
lista_feminino = []

for classe in valores_classes_idade:
    for sexo in valores_sexo:
        if sexo == 1:
            lista_masculino.append(dados['IMC'][(dados['VB01001'] == sexo) & (dados['CLASSE_IDADE'] == classe)].mode()[0])
        elif sexo == 2:
            lista_feminino.append(dados['IMC'][(dados['VB01001'] == sexo) & (dados['CLASSE_IDADE'] == classe)].mode()[0])

In [ ]:
df_moda_imc_sexo_idade = pd.DataFrame([lista_masculino, lista_feminino]).T
df_moda_imc_sexo_idade.index = [valores_classes_idade]
df_moda_imc_sexo_idade.columns = ['Masculino', 'Feminino']
df_moda_imc_sexo_idade

#### Alguns gráficos

##### Histograma - Distribuição geral dos IMCs

No caso dos IMCs, temos a ocorrência de uma distribuição multimodal (11 valores). Assim, foram mostrados apenas alguns dos valores.

In [ ]:
ax = sns.histplot(data = dados, x = 'IMC')
ax.figure.set_size_inches((13, 6))
ax.set_title('Distribuição dos IMCs', fontsize = 22)
ax.set_xlabel('Imc', fontsize = 12)
ax.set_ylabel('Quantidade de ocorrências', fontsize = 12)

# linha com a média dos pesos
ax.axvline(x = dados['IMC'].mean(),
           label = 'Média',
           linewidth = 2,
           color = '#000000')

# linha com a mediana dos pesos
ax.axvline(x = dados['IMC'].median(),
           label = 'Mediana',
           linewidth = 2,
           linestyle = '--',
           color = '#7FFF00')

#linha com a moda 0
ax.axvline(x = dados['IMC'].mode()[0],
           label = 'Menor valor entre as modas',
           linewidth = 2,
           linestyle = '-.',
           color = '#DB7093')

# linha com a moda 5
ax.axvline(x = dados['IMC'].mode()[5],
           label = 'Mediana das Modas',
           linewidth = 2,
           linestyle = '-.',
           color = '#CD5C5C')

# linha com a moda 1
ax.axvline(x = dados['IMC'].mode()[10],
           label = 'Maior valor entre as Modas',
           linewidth = 2,
           linestyle = '-.',
           color = '#DC143C')

# intervalo de confiança da distribuição dos pesos
xmin = media_imc - desvio_padrao_imc
xmax = media_imc + desvio_padrao_imc
# xmin = norm.interval(alpha = 0.95, loc = media, scale = (std / np.sqrt(n)))[0]
# xmax = norm.interval(alpha = 0.95, loc = media, scale = (std / np.sqrt(n)))[1]
# área do desvio padrão
ax.axvspan(xmin, xmax, alpha=0.3, color='gray', label='Desvio Padrão')

ax.legend(title = 'Medidas')
ax;

##### Boxplot - distribuição dos IMCs de acordo com a classe de idade

In [ ]:
ax = sns.boxplot(x = dados['CLASSE_IDADE'], y = dados['IMC'], palette = 'winter_r')
ax.figure.set_size_inches((13,6))
ax.set_xlabel('Classe de idade', fontsize = 14)
ax.set_ylabel('IMC', fontsize = 14)
ax;

##### Boxplot - Distribuição dos IMCs de acordo com o sexo e classe de idade

In [ ]:
ax = sns.boxplot(data = dados,
                 x = 'CLASSE_IDADE',
                 y = 'IMC',
                 hue = 'VB01001', palette = 'Accent_r')
ax.figure.set_size_inches((13,6))
ax.set_xlabel('Classe de idade', fontsize = 14)
ax.set_ylabel('IMC', fontsize = 14)
handles, _ = ax.get_legend_handles_labels()          
ax.legend(handles, ["Masculino", "Feminino"], loc="best")
ax;

##### Boxplot - Distribuição dos IMCs de acordo com o sexo

In [ ]:
ax = sns.boxplot(x = dados['VB01001'], y = dados['IMC'])
ax.set_xticklabels(labels = ['Masculino', 'Feminino'])
ax.set_xlabel('Sexo', fontsize = 14, labelpad = 20)
ax.set_ylabel('IMC', fontsize = 14, labelpad = 20)
ax.figure.set_size_inches((12,6))

##### Boxplot - Distribuição dos IMCs de acordo com a raça

In [ ]:
ax = sns.boxplot(x = dados['VB01002'], y = dados['IMC'])
ax.set_xticklabels(labels = raca)
ax.set_xlabel('Raça', fontsize = 14, labelpad = 20)
ax.set_ylabel('IMC', fontsize = 14, labelpad = 20)
ax.figure.set_size_inches((12,6))

### Classe do IMC

#### Criando a coluna CLASSES_IMC

In [ ]:
classes = [0, 18.5, 25, 30, 40, 80]
labels = ['MAGREZA', 'NORMAL', 'SOBREPESO', 'OBESIDADE', 'OBESIDADE GRAVE']

dados['CLASSE_IMC'] = pd.cut(x = dados['IMC'], bins = classes, labels = labels, include_lowest = True)

#### Quantidade de ocorrências das classes do IMC

In [ ]:
counts_classe_imc = pd.DataFrame(dados.CLASSE_IMC.value_counts(ascending = True))
counts_classe_imc.columns = ['CLASSE_IMC']
counts_classe_imc

In [ ]:
ax = sns.countplot(data = dados, y = 'CLASSE_IMC')
ax.figure.set_size_inches((9,5))
ax.set_title('Número de ocorrências das classes do IMC', fontsize = 22, pad = 20)
ax.set_xlabel('Número de ocorrências', fontsize = 14, labelpad = 20)
ax.set_ylabel('Classe do IMC', fontsize = 14, labelpad = 20)
ax.grid()
ax;

#### Quantidade de alunos <ins>abaixo e acima do peso ideal</ins>

In [ ]:
abaixo = dados[dados['IMC'] < 18.5]['CLASSE_IMC'].count()
normal = dados[(dados['IMC'] >= 18.5) & ((dados['IMC'] < 25))]['CLASSE_IMC'].count()
acima = dados[dados['IMC'] >= 25]['CLASSE_IMC'].count()

In [ ]:
df_num_absolutos_imc = pd.DataFrame([abaixo, normal, acima])
df_num_absolutos_imc.index = ['Abaixo do peso', 'Peso normal', 'Acima do peso']
df_num_absolutos_imc.columns = ['Quantidade']
df_num_absolutos_imc

In [ ]:
porc_abaixo = (abaixo / len(dados.CLASSE_IMC) ) * 100
porc_normal = (normal / len(dados.CLASSE_IMC) ) * 100
porc_acima = (acima / len(dados.CLASSE_IMC) ) * 100

In [ ]:
df_porc_imc = pd.DataFrame([porc_abaixo, porc_normal, porc_acima])
df_porc_imc.index = ['Abaixo do peso', 'Peso normal', 'Acima do peso']
df_porc_imc.columns = ['Porcentagem']
df_porc_imc

In [ ]:
fig = px.pie(df_num_absolutos_imc,
             values='Quantidade',
             names= df_num_absolutos_imc.index,
             labels = ['Abaixo do peso', 'Peso normal', 'Acima do peso'],
             title='Porcentagem e Quantidade de alunos abaixo e acima do peso e com peso normal')
fig.show()

#### <ins>Classes de IMC</ins> de acordo com o <ins>sexo</ins>

##### Números absolutos de cada classe do IMC, de acordo com o sexo

In [ ]:
imc_sexo_abs = pd.crosstab(index = dados['CLASSE_IMC'], columns = dados['VB01001'])
imc_sexo_abs.columns = ['Masculino', 'Feminino']
imc_sexo_abs.index.name = 'Classe IMC'
imc_sexo_abs

##### Porcentagens de cada classe do IMC, de acordo com o sexo

In [ ]:
imc_sexo_porc = pd.crosstab(index = dados['CLASSE_IMC'], columns = dados['VB01001'],
                       normalize = 'index')
imc_sexo_porc.columns = ['Masculino', 'Feminino']
imc_sexo_porc[['Masculino', 'Feminino']] = imc_sexo_porc[['Masculino', 'Feminino']]  * 100
imc_sexo_porc.index.name = 'Classe IMC'
imc_sexo_porc

In [ ]:
imc_sexo_porc.iloc[0]

In [ ]:
fig = px.pie(imc_sexo_porc,
             values = imc_sexo_porc.iloc[0],
             names= imc_sexo_porc.columns,
             title='Porcentagem de alunos, por sexo, abaixo do peso ideal')
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=5, specs=[[{"type": "pie"}, {"type": "pie"},
                                            {"type": "pie"}, {"type": "pie"}, {"type": "pie"}]])

fig.add_trace(go.Pie(
     values=imc_sexo_porc.iloc[0],
     labels=['Masculino', 'Feminino'],
     name="Sexo",
     title = imc_sexo_porc.iloc[0].name), 
     row=1, col=1)

fig.add_trace(go.Pie(
     values=imc_sexo_porc.iloc[1],
     labels=['Masculino', 'Feminino'],
     name="Sexo",
     title = imc_sexo_porc.iloc[1].name),
    row=1, col=2)

fig.add_trace(go.Pie(
     values=imc_sexo_porc.iloc[2],
     labels=['Masculino', 'Feminino'],
     name="Sexo",
     title = imc_sexo_porc.iloc[2].name),
    row=1, col=3)

fig.add_trace(go.Pie(
     values=imc_sexo_porc.iloc[3],
     labels=['Masculino', 'Feminino'],
     name="Sexo",
     title = imc_sexo_porc.iloc[3].name),
    row=1, col=4)

fig.add_trace(go.Pie(
     values=imc_sexo_porc.iloc[4],
     labels=['Masculino', 'Feminino'],
     name="Sexo",
    domain=dict(x=[0.5, 1.0]),
     title = imc_sexo_porc.iloc[4].name),
    row=1, col=5)

fig.update_layout(height=300, showlegend=True,
                  title = 'Porcentagens de alunos em cada classe de IMC, de acordo com o sexo')
go.Layout()
fig.show()

In [ ]:
imc_sexo_porc.iloc[0].name

#### <ins>Classes de IMC</ins> de acordo com a <ins>raça</ins>

In [ ]:
raca_abs

##### Números absolutos

In [ ]:
imc_raca_abs = pd.crosstab(index = dados['CLASSE_IMC'], columns = dados['VB01002'])
imc_raca_abs.columns = raca
imc_raca_abs.index.name = 'Classe IMC'
imc_raca_abs

##### Porcentagens

In [ ]:
imc_raca_abs = pd.crosstab(index = dados['CLASSE_IMC'], columns = dados['VB01002'],
                           normalize = 'index')
imc_raca_abs.columns = raca
colunas = imc_raca_abs.columns
imc_raca_abs[colunas] = imc_raca_abs[colunas] * 100
imc_raca_abs.index.name = 'Classe IMC'
print('Normalização por linha')
imc_raca_abs

##### Distribuição de IMC por raça

In [ ]:
imc_raca = dados.query('VB01002 != 99')

ax = sns.boxplot(x = imc_raca['VB01002'], y = dados['IMC'])
ax.set_xticklabels(labels = colunas[:5])
ax.set_xlabel('Raça', fontsize = 14, labelpad = 20)
ax.set_ylabel('IMC', fontsize = 14, labelpad = 20)
ax.figure.set_size_inches((12,6))

### Grau de satisfação com o corpo

In [ ]:
# número de respostas para o grau de satisfação com o corpo
sat = dados[(dados['VB11007'] == 1) | (dados['VB11007'] == 2)]['VB11007'].sum()
indiferente = dados[dados['VB11007'] == 3]['VB11007'].sum()
ins = dados[(dados['VB11007'] == 4) | (dados['VB11007'] == 5)]['VB11007'].sum()
n_informado = dados[dados['VB11007'] == 99]['VB11007'].sum()

In [ ]:
# Distribuição das respostas sobre satisfação com o corpo
df_satisfacao_corpo = pd.DataFrame([sat, indiferente, ins, n_informado])
df_satisfacao_corpo.index = ['Satisfeitos ou muito satisfeitos', 'Indiferente',
                             'Insatisfeitos ou muito insatisfeitos', 'Não informado']
df_satisfacao_corpo.index.name = 'Grau de satisfação'
df_satisfacao_corpo.columns = ['Nº de respostas']
df_satisfacao_corpo

In [ ]:
fig = px.bar(data_frame = df_satisfacao_corpo,x = df_satisfacao_corpo.index, y = 'Nº de respostas',
             color = df_satisfacao_corpo.index, color_discrete_sequence=px.colors.qualitative.Dark2,
             title = 'Número de respostas para cada grau de satisfação com o corpo')
fig.update_layout(font_family="Rockwell", legend=dict(
                  title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"))
fig.show()

In [ ]:
plt.figure(figsize = (15,5))
fig = px.pie(df_satisfacao_corpo,
             values='Nº de respostas',
             names= df_satisfacao_corpo.index,
             labels = df_satisfacao_corpo.index,
             title='Números e porcentagens de respostas sobre o grau de satisfação com o corpo')
fig.show()

Apesar de menor, o número de alunos insatisfeitos ou muito insatisfeitos com o próprio corpo equivale a quase 70% dos que se sentem satisfeitos ou muito satisfeitos com o próprio corpo.

# VERIFICAÇÃO DE ALGUMAS HIPÓTESES

## A média do IMC das pessoas insatisfeitas ou muito insatisfeitas com seu corpo está abaixo ou acima do IMC considerado normal (entre 18,5 e 24,9).

In [ ]:
ins = dados[(dados['VB11007'] == 4) | (dados['VB11007'] == 5)]

In [ ]:
ins['IMC'].mean()

Acima verificamos que a média do IMC das pessoas insatisfeitas ou muito insatisfeitas com o próprio corpo é de aproximadamente 23, simplismente dentro do que é considerado um peso normal de acordo com a medida proposta.

In [ ]:
n = len(ins['IMC'])
media = ins['IMC'].mean()
std = ins['IMC'].std()

In [ ]:
norm.interval(alpha = 0.95, loc = media, scale = std / np.sqrt(n))

Se repetirmos o experimento várias vezes, em 95% delas a média do IMC populacional de alunos insatisfeitos ou muito insatisfeitos com o próprio corpo estará entre no intervalo entre 22.69 e 23.06.

## Há grande diferença nas porcentagens de alunos satisfeitos ou muito satisfeitos, em relação ao corpo, por raça/cor.

In [ ]:
df = pd.crosstab(index = dados['VB01002'], columns = dados['VB11007'])
df.index = raca
df.columns = ['Muito satisfeito(a)', 'Satisfeito(a)','Indiferente','Insatisfeito(a)',
              'Muito insatisfeito(a)', 'Não informado']

In [ ]:
df['Total'] = df.sum(axis = 1)
df

### Porcentagem de alunos satisfeitos ou muito satisfeitos para cada cor/raça

In [ ]:
# quantidade de respostas de satisfação para cada raça/cor
sat = dados[(dados['VB11007'] == 1) | (dados['VB11007'] == 2)]['VB01002'].value_counts().sort_index()
sat

In [ ]:
satisfacao_cada_raca = pd.DataFrame(sat)
satisfacao_cada_raca.index = raca
satisfacao_cada_raca.index.name = 'Raça'
satisfacao_cada_raca.columns = ['Nº de alunos satisfeitos']
satisfacao_cada_raca

In [ ]:
satisfacao_cada_raca['Respondentes x Raça'] = raca_abs['Nº de respondentes']

In [ ]:
satisfacao_cada_raca

In [ ]:
valores = dados['VB01002'].unique()
valores.sort()
valores

In [ ]:
for i in valores:
    satisfacao_cada_raca['Porcentagem_raca'] = (satisfacao_cada_raca['Nº de alunos satisfeitos'] / satisfacao_cada_raca['Respondentes x Raça']) * 100
    break

In [ ]:
satisfacao_cada_raca

In [ ]:
# gráfico de barras
plt.figure(figsize = (15,6));
fig = px.bar(data_frame = satisfacao_cada_raca,
                x = satisfacao_cada_raca.index,
                y = 'Porcentagem_raca',
                title = 'Porcentagem, de cada raça, de alunos satisfeitos ou muito satisfeitos com o corpo',
                color = satisfacao_cada_raca.index,
                color_discrete_sequence=px.colors.qualitative.Dark2)
fig.show()

O gráfico acima nos mostra a porcentagem de alunos de cada cor/raça que estavam satisfeitos e muito satisfeitos com o corpo.

Podemos perceber que não há grande diferença entre porcentagens.

A maior diferença entre as porcentagens fica em aproximadamente 5.25%, entre a cor/raça preta (73.80% estavam satisfeitos ou muito satisfeitos) e a cor/raça amarela (67.55% estavam satisfeitos ou muito satisfeitos).

## As alunas tendem a se sentir insatisfeitas ou muito insatisfeitas com o próprio corpo mais do que os alunos.

In [ ]:
# alunos que estavam insatisfeitos ou muito insatisfeitos
insat = dados[(dados['VB11007'] == 4) | (dados['VB11007'] == 5)]

In [ ]:
# quantidade de alunos e alunas insatisfeitos ou muito insatisfeitos
num_insat = insat['VB01001'].value_counts().sort_index()

In [ ]:
# quantidade de alunos, de cada sexo, que responderam a pesquisa
num_alunos_alunas = dados['VB01001'].value_counts()

In [ ]:
df_alunos_alunas = pd.DataFrame(num_alunos_alunas)
df_alunos_alunas.index = ['Masculino', 'Feminino']
df_alunos_alunas.columns = ['Nº respondentes']

In [ ]:
df_alunos_alunas['Muito OU insatisfeitos'] = num_insat.values

In [ ]:
df_alunos_alunas

In [ ]:
plt.figure(figsize = (15,6))
px.pie(data_frame = df_alunos_alunas,
       values = 'Muito OU insatisfeitos',
       names = df_alunos_alunas.index,
       title = 'Porcentagem, por sexo, dos alunos(as) insatisfeitos(as) ou muito insatisfeitos(as) com o corpo',
       color = df_alunos_alunas.index,
       color_discrete_sequence=px.colors.qualitative.Pastel)

Dos 3040 alunos e alunas que se sentem insatisfeitos ou muito insatisfeitos em relação ao corpo, vemos que as meninas representam 65.6% desse número, contra 34.4% dos meninos. 

Temos o total de alunos e alunas, que responderam o questionário, praticamente igual, sendo 8287 para alunos e 8269 alunas. Com isso, parece que as alunas tendem a se sentir mais pressionadas em relação ao seu corpo, talvez por pressão midiática, pela sociedade machista ou por outros motivos que podem influenciar nesse sentimento.

## O fato de o aluno(a) ter ficado "magoado, incomodado, aborrecido, ofendido ou humilhado" com frequência quando seus colegas "esculacharam, zoaram, mangaram, intimidaram ou caçoaram" pode ter levado-o a ter o IMC abaixo ou acima de um IMC normal .

**Pergunta VB07007:**
<ins>"NOS ÚLTIMOS 30 DIAS, com que frequência algum dos seus colegas de escola te esculacharam, zoaram, mangaram, intimidaram ou caçoaram tanto que você ficou magoado, incomodado, aborrecido, ofendido ou humilhado?"</ins>


Vamos analisar, primeiramente, as quantidades e as porcentagens das respostas.

### Verificando a quantidade de cada resposta, exceto a resposta 99 ("Não informado") :

In [ ]:
dados_intimidacao = pd.DataFrame(dados['VB07007'][dados['VB07007'] != 99].value_counts())
dados_intimidacao.index = ['Nunca', 'Raramente', 'Às vezes', 'Na maior parte do tempo', 'Sempre']
dados_intimidacao.index.name = 'Resposta'
dados_intimidacao.columns = ['Quantidade']
dados_intimidacao

In [ ]:
fig = px.bar(data_frame = dados_intimidacao, 
       x = dados_intimidacao.index, 
       y = dados_intimidacao['Quantidade'],
      color = dados_intimidacao.index, 
      )
fig.update_xaxes(title_text='Resposta', title_font = {"size": 14}, title_standoff = 30)
fig.update_yaxes(title_text='Quantidade', title_font = {"size": 14}, title_standoff = 15)
fig.update_layout(title_text= 'Número de respostas sobre intimidação dos colegas nos últimos 30 dias',
                 title_font = {"size": 18})
fig.show()

O gráfico acima nos mostra que as respostas "Na maior parte do tempo" e "Sempre" possuem, cada uma, menos de 700 ocorrências, enquanto 9147 alunos declararam que nunca sofreram nenhum tipo de "intimidação" de seus colegas nos últimos 30 dias. Veremos agora a porcentagem representada por cada resposta.

### Verificando a porcentagem, de cada resposta, exceto a resposta 99 ("Não informado") :

In [ ]:
fig = px.pie(data_frame = dados_intimidacao,
             names = dados_intimidacao.index,
             values = 'Quantidade')
fig.update_layout(title_text= 'Porcentagem das respostas sobre intimidação dos colegas nos últimos 30 dias',
                 title_font = {"size": 18})
fig.show()

### Para a análise da hipótese, vamos assumir a possibilidade de que essa "perseguição" ao aluno pode ter se prolongado por mais de 30 dias, para que haja um intervalo maior em uma mudança regular em seu peso e, consequentemente, no seu IMC (entre 18,5 e 24,9).

In [ ]:
intimidacao = dados[dados['VB07007'] != 99].copy()

In [ ]:
# dados dos alunos que responderam "Na maior parte do tempo" (resposta 4) e 
# "Sempre" (resposta 5).
intimidacao_maior_tempo_sempre = intimidacao.query("(VB07007 == 4) | (VB07007 == 5)")
intimidacao_maior_tempo_sempre['IMC'].mean()

In [ ]:
# dados dos alunos que responderam "As vezes" (resposta 3).
intimidacao_as_vezes = dados.query("(VB07007 == 3)")
intimidacao_as_vezes['IMC'].mean()

In [ ]:
# dados dos alunos que responderam "Nunca" (resposta 1) e "Raramente" (resposta 2).
intimidacao_nunca_raramente = dados.query("(VB07007 == 1) | (VB07007 == 2)")
intimidacao_nunca_raramente['IMC'].mean()

Com as médias calculadas acima não podemos afirmar que sofreram "intimidação" com maior frequência tendem a possuir um IMC abaixo ou acima do normal. Agora vamos visualizar a distribuição dos dados para termos maior grau de certeza:

In [ ]:
respostas = {1: 'Nunca', 2: 'Raramente', 3: 'As vezes', 4: 'Na maior parte do tempo', 5: 'Sempre'}
intimidacao['VB07007'] = intimidacao['VB07007'].map(respostas)
intimidacao.rename(columns = {'VB07007': 'Intimidação nos últimos 30 dias'}, inplace = True)

In [ ]:
fig = px.box(data_frame = intimidacao, x = 'Intimidação nos últimos 30 dias', y = 'IMC', 
      color = 'Intimidação nos últimos 30 dias',
      title = 'Distribuição do IMC dos alunos que sofreram "intimidação" as vezes')
fig.show()

Como podemos perceber na visualização acima, parece que a frequência com que o aluno(a) sofre algum tipo de "intimidação" não influencia no seu IMC.
Com isso, rejeitamos a nossa hipótese nula.